<a href="https://colab.research.google.com/github/ViacheslavCRBP/pythonProject/blob/Rec_sistem_course/HW_PyTorch_07_ViacheslavS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание №7. Рекуррентные сети для обработки последовательностей

In [1]:
!pip install stop-words pymorphy2

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 21.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32898 sha256=aa30e8e954b80d893d4e3fbd37a21e531b751f8ec0293f71163227ce53eb103e
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=fbc6bbc186526392d5bab05f2d9df04481b4b7019bb0f2c620fa3da596059fb0
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built stop-words docopt


In [2]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 9.9 MB/s eta 0:00:00


In [3]:
#предобработка
from string import punctuation

from pymorphy2 import MorphAnalyzer
import re
import nltk
nltk.download('wordnet')
nltk.download("punkt")
nltk.download('stopwords')
nltk.download('omw-1.4')

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

import torchmetrics

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [4]:
max_words = 20000
max_len = 10
num_classes = 1

# обучение
epochs = 10
batch_size = 512
print_batch_n = 100

## 0. Загрузка данных

In [5]:
!pip install opendatasets

In [6]:
import opendatasets as od
import pandas

od.download(
    "https://drive.google.com/file/d/1LRPqfo8kI-9mIU59dYxINb7cjhR3uYc_/view?usp=sharing")

3806066it [00:00, 1029258404.13it/s]

Downloaded and unzipped to  ./data-20231118T080219Z-001


In [9]:
# пути к данным
root_data_path = 'https://drive.google.com/file/d/1TZX5JVxeDmS3ndZHrZq2jjFFuaeSOaNN/view?usp=sharing'

dataset_train  = root_data_path + '/train.csv'
dataset_test  = root_data_path + '/test.csv'

In [11]:
#загрузка данных
data_train = pd.read_csv(dataset_train, encoding='utf-8')
data_test = pd.read_csv(dataset_test, encoding='utf-8')

In [12]:

#тренировочный датасет
data_train.head()

,"<!DOCTYPE html><html><head><meta name=""google"" content=""notranslate""><meta http-equiv=""X-UA-Compatible"" content=""IE=edge;""><style nonce=""6u0R4zQRPuTwcjTSc2Ydxw"">@font-face{font-family:'Roboto';font-style:italic;font-weight:400;src:url(//fonts.gstatic.com/s/roboto/v18/KFOkCnqEu92Fr1Mu51xIIzc.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:300;src:url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmSU5fBBc9.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:400;src:url(//fonts.gstatic.com/s/roboto/v18/KFOmCnqEu92Fr1Mu4mxP.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:500;src:url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmEU9fBBc9.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:700;src:url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmWUlfBBc9.ttf)format('truetype');}</style><meta name=""referrer"" content=""origin""><title>archive.zip - Google Drive</title><meta property=""og:title"" content=""archive.zip""><meta property=""og:type"" content=""article""><meta property=""og:site_name"" content=""Google Docs""><meta property=""og:url"" content=""https://drive.google.com/file/d/1TZX5JVxeDmS3ndZHrZq2jjFFuaeSOaNN/view?usp=sharing/train.csv&amp;usp=embed_facebook""><link rel=""shortcut icon"" href=""https://ssl.gstatic.com/images/branding/product/1x/drive_2020q4_32dp.png""><link rel=""stylesheet"" href=""https://fonts.googleapis.com/css?family=Google+Sans:300",400,500,"700"" nonce=""6u0R4zQRPuTwcjTSc2Ydxw""><link rel=""stylesheet"" href=""https://www.gstatic.com/_/apps-fileview/_/ss/k=apps-fileview.v.1fJi0INB_Zs.L.X.O/am=IAw/d=0/rs=AO0039uu04tY8I3X8IZ1hjjX0tVRzV2stw"" nonce=""6u0R4zQRPuTwcjTSc2Ydxw""><script nonce=""8sh3AAfyCOdpAdtyH4PeVw"">_docs_flag_initialData={""docs-ails"":""docs_cold""","docs-fwds:""docs_sdf""","docs-crs:""docs_crs_nl""",docs-fe-re:2,docs-fl:1,docs-l1lc:2,"docs-l1lm:""DLS""",...,0.89,0.90,0.91,1.70,72175901].1,[[null.2,null.215,null.216,"https://www.gstatic.com/og/_/js/k=og.qtm.en_US.UfLhtnONdTE.es5.O/rt=j/m=qabr,q_dnp,qapid,q_dg/exm=qaaw,qadd,qaid,qein,qhaw,qhba,qhbr,qhch,qhga,qhid,qhin/d=1/ed=1/rs=AA2YrTuLrDBZ32t76yhZnR9MRGEGUJqy3Q]]]]",};this.gbar_=this.gbar_||{};(function(_){var window=this;
0,try{,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"_._F_toggles_initialize=function(a){(""undefine...",_._F_toggles_initialize)([]);,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,/*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Copyright The Closure Library Authors.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SPDX-License-Identifier: Apache-2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
#тестовый датасет
data_test.head()

,"<!DOCTYPE html><html><head><meta name=""google"" content=""notranslate""><meta http-equiv=""X-UA-Compatible"" content=""IE=edge;""><style nonce=""b3Xz2UaewweGsba8I5Uf5A"">@font-face{font-family:'Roboto';font-style:italic;font-weight:400;src:url(//fonts.gstatic.com/s/roboto/v18/KFOkCnqEu92Fr1Mu51xIIzc.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:300;src:url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmSU5fBBc9.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:400;src:url(//fonts.gstatic.com/s/roboto/v18/KFOmCnqEu92Fr1Mu4mxP.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:500;src:url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmEU9fBBc9.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:700;src:url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmWUlfBBc9.ttf)format('truetype');}</style><meta name=""referrer"" content=""origin""><title>archive.zip - Google Drive</title><meta property=""og:title"" content=""archive.zip""><meta property=""og:type"" content=""article""><meta property=""og:site_name"" content=""Google Docs""><meta property=""og:url"" content=""https://drive.google.com/file/d/1TZX5JVxeDmS3ndZHrZq2jjFFuaeSOaNN/view?usp=sharing/test.csv&amp;usp=embed_facebook""><link rel=""shortcut icon"" href=""https://ssl.gstatic.com/images/branding/product/1x/drive_2020q4_32dp.png""><link rel=""stylesheet"" href=""https://fonts.googleapis.com/css?family=Google+Sans:300",400,500,"700"" nonce=""b3Xz2UaewweGsba8I5Uf5A""><link rel=""stylesheet"" href=""https://www.gstatic.com/_/apps-fileview/_/ss/k=apps-fileview.v.1fJi0INB_Zs.L.X.O/am=IAw/d=0/rs=AO0039uu04tY8I3X8IZ1hjjX0tVRzV2stw"" nonce=""b3Xz2UaewweGsba8I5Uf5A""><script nonce=""EThrr0whNN4_ckq7i4uaFQ"">_docs_flag_initialData={""docs-ails"":""docs_cold""","docs-fwds:""docs_sdf""","docs-crs:""docs_crs_nl""",docs-fe-re:2,docs-fl:1,docs-l1lc:2,"docs-l1lm:""DLS""",...,0.90,0.91,0.92,1.69,72175901].1,[[null.2,null.215,null.216,"https://www.gstatic.com/og/_/js/k=og.qtm.en_US.UfLhtnONdTE.es5.O/rt=j/m=qabr,q_dnp,qapid,q_dg/exm=qaaw,qadd,qaid,qein,qhaw,qhba,qhbr,qhch,qhga,qhid,qhin/d=1/ed=1/rs=AA2YrTuLrDBZ32t76yhZnR9MRGEGUJqy3Q]]]]",};this.gbar_=this.gbar_||{};(function(_){var window=this;
0,try{,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"_._F_toggles_initialize=function(a){(""undefine...",_._F_toggles_initialize)([]);,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,/*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Copyright The Closure Library Authors.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SPDX-License-Identifier: Apache-2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
#разделение тренировочного датасета на обучающий и валидационный
df_train, df_valid = train_test_split(data_train, test_size=0.3, random_state=42)

In [15]:
#датасет для обучения модели
df_train.head()

,"<!DOCTYPE html><html><head><meta name=""google"" content=""notranslate""><meta http-equiv=""X-UA-Compatible"" content=""IE=edge;""><style nonce=""6u0R4zQRPuTwcjTSc2Ydxw"">@font-face{font-family:'Roboto';font-style:italic;font-weight:400;src:url(//fonts.gstatic.com/s/roboto/v18/KFOkCnqEu92Fr1Mu51xIIzc.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:300;src:url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmSU5fBBc9.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:400;src:url(//fonts.gstatic.com/s/roboto/v18/KFOmCnqEu92Fr1Mu4mxP.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:500;src:url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmEU9fBBc9.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:700;src:url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmWUlfBBc9.ttf)format('truetype');}</style><meta name=""referrer"" content=""origin""><title>archive.zip - Google Drive</title><meta property=""og:title"" content=""archive.zip""><meta property=""og:type"" content=""article""><meta property=""og:site_name"" content=""Google Docs""><meta property=""og:url"" content=""https://drive.google.com/file/d/1TZX5JVxeDmS3ndZHrZq2jjFFuaeSOaNN/view?usp=sharing/train.csv&amp;usp=embed_facebook""><link rel=""shortcut icon"" href=""https://ssl.gstatic.com/images/branding/product/1x/drive_2020q4_32dp.png""><link rel=""stylesheet"" href=""https://fonts.googleapis.com/css?family=Google+Sans:300",400,500,"700"" nonce=""6u0R4zQRPuTwcjTSc2Ydxw""><link rel=""stylesheet"" href=""https://www.gstatic.com/_/apps-fileview/_/ss/k=apps-fileview.v.1fJi0INB_Zs.L.X.O/am=IAw/d=0/rs=AO0039uu04tY8I3X8IZ1hjjX0tVRzV2stw"" nonce=""6u0R4zQRPuTwcjTSc2Ydxw""><script nonce=""8sh3AAfyCOdpAdtyH4PeVw"">_docs_flag_initialData={""docs-ails"":""docs_cold""","docs-fwds:""docs_sdf""","docs-crs:""docs_crs_nl""",docs-fe-re:2,docs-fl:1,docs-l1lc:2,"docs-l1lm:""DLS""",...,0.89,0.90,0.91,1.70,72175901].1,[[null.2,null.215,null.216,"https://www.gstatic.com/og/_/js/k=og.qtm.en_US.UfLhtnONdTE.es5.O/rt=j/m=qabr,q_dnp,qapid,q_dg/exm=qaaw,qadd,qaid,qein,qhaw,qhba,qhbr,qhch,qhga,qhid,qhin/d=1/ed=1/rs=AA2YrTuLrDBZ32t76yhZnR9MRGEGUJqy3Q]]]]",};this.gbar_=this.gbar_||{};(function(_){var window=this;
30,function(g){var h=void 0;try{h=g.then}catch(k)...,g):this.B(g)};e.prototype.A=function(g){this.D(2,g)};e.prototype.B=function(g){this.D(1,g)};e.prototype.D=function(g,"h){if(0!=this.i)throw Error(""i`""+g+""`""+h+""`""+t...",1)};e.prototype.H=function(){if(this.C)return!...,h=,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,}catch(e){_._DumpException(e)},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,_.ke=function(a){this.J=_.v(a)};_.z(_.ke,_.S);,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,_.ya=function(a){if(!_.va)return _.wa(a);for(v...,c=0,d=a.length-10240;c<d;)b+=String.fromCharCode.a...,a.subarray(c,c+=10240));b+=String.fromCharCode.apply(null,c?a.subarray(c):a);return btoa(b)};_.Aa=functi...,_.Ca)(a);1!==(b&1)&&(Object.isFrozen(a)&&(a=_....,(0,_.Da)(a,b|1))};Ga=function(){var a=[];(0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,f.j(k):this.j.push(k);this.C=!0};e.resolve=c;e...,k){k(g)})};e.race=function(g){return new e(fun...,k){for(var m=_.Eb(g),n=m.next();!n.done;n=m.next())c(n.value).Id(h,k)})};e.all=function(g){var h=_.Eb(g),k=h.next();return k.done?c([]):new e(function(m,n){function q(x){return function(B){u[x]=B;r--...,r=0;do u.push(void 0),r++,c(k.value).Id(q(u.length-1),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
df_train['label'].value_counts()

KeyError: ignored

In [23]:

# датасет для валидации модели
# df_valid.head()

In [24]:
# df_valid['label'].value_counts()

## 1. Предобработка данных

In [22]:
#множество стоп-слов
sw = set(stopwords.words('english'))
print(sw)

{"doesn't", "aren't", 'is', 'yourselves', "it's", 'once', 'between', 'ain', 'o', 'have', 'only', 'themselves', 'more', 'because', 'which', 'below', 'should', 'he', 'who', "shan't", 'm', 'further', 'doesn', "you'll", 'does', 'were', 'while', 'the', 'himself', 'over', 'no', 'she', 'some', 'yourself', 'your', 'do', 't', 'are', 'their', 'for', 'them', 'out', 'be', 've', "you're", 'having', 'hadn', "you'd", 'has', 'i', "won't", 'being', 'own', 'hasn', 'any', 'there', 'here', 'through', 'if', 'did', 'at', "didn't", 'during', 'am', 'just', 'most', 'so', "shouldn't", 'what', 'a', "couldn't", 'yours', 'her', 'mightn', 'an', 'wouldn', 'didn', 'of', 'theirs', 'or', 'then', "wouldn't", 'why', 'on', 'this', 'such', 'from', 'until', "don't", "should've", 'y', 'been', "isn't", 'shouldn', 'with', 'very', 'under', 'all', 're', 'needn', 'shan', 'weren', 'ourselves', 'herself', 'was', 'nor', 'in', 'by', 'wasn', 'my', 'but', 'it', 'above', "needn't", 'these', 'hers', 'other', 'll', 'mustn', 'about', 'too'

In [25]:
#пунктуация
exclude = set(punctuation)
print(exclude)

{';', ')', '^', ':', '@', '.', '/', '*', '_', ']', '&', '%', '=', '"', "'", '$', '{', '<', '?', '(', '`', '|', '+', '~', '[', '#', '-', '>', '\\', '}', ',', '!'}


In [26]:
#морфологический анализ для английского текста
lemmatizer = WordNetLemmatizer()

#пример работы
lemmatizer.lemmatize('dogs'), lemmatizer.lemmatize('says')

('dog', 'say')

In [27]:
#функция преобразования текста
def preprocess_text(txt):
    #преобразуем входные данные в строку
    txt = str(txt)
    #удяляем знаки пунктуации
    txt = "".join(c for c in txt if c not in exclude)
    #приводим все к нижнему регистру
    txt = txt.lower()
    #лематизация и удаление стоп-слов
    txt = [lemmatizer.lemmatize(w) for w in txt.split() if w not in sw]
    return " ".join(txt)

In [31]:
#преобразование текста в данных
from tqdm import tqdm
tqdm.pandas()

# df_train['tweet'] = df_train['tweet'].progress_apply(preprocess_text)
# df_valid['tweet'] = df_valid['tweet'].progress_apply(preprocess_text)
# data_test['tweet'] = data_test['tweet'].progress_apply(preprocess_text)

In [33]:
#соединение всех обучающих данных в одну строку и приведение к нижнему регистру
# train_corpus = " ".join(df_train["tweet"])
# train_corpus = train_corpus.lower()

In [34]:
#токенезация (разбиваем строку на слова)
tokens = word_tokenize(train_corpus)
tokens[:10]

NameError: ignored

In [ ]:
#отфильтруем токены (проверяем, чтобы все символы были или буквы или цифры)
tokens_filtered = [word for word in tokens if word.isalnum()]
print(tokens_filtered)

In [ ]:
#соберем max_words наиболее частых токенов

#частотное распределение: как часто встречается каждый токен в tokens_filtered
dist = FreqDist(tokens_filtered)
dist

In [ ]:
#берем max_words наиболее часто встречающихся токенов
#(создаем упорядоченные пары (слово, частота) в количестве max_words штук и берем первые элементы пар)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]  # -1 - padding
len(tokens_filtered_top), tokens_filtered_top[:10]

In [ ]:
#словарь слово : индекс из tokens_filtered_top
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top,1)).items()}
vocabulary

In [ ]:
#преобразование текста в последовательность
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())  #токенизация
    tokens_filtered = [word for word in tokens if word.isalnum()] #фильтруем (только буквы и цифры)
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word]) #если слово в топе токенов, то добавляем его индекс в результат

    padding = [0] * (maxlen-len(result)) #нули дополняющие до maxlen
    return result[-maxlen:] + padding

In [ ]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["tweet"]], dtype=np.int32)
x_valid = np.asarray([text_to_sequence(text, max_len) for text in df_valid["tweet"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in data_test["tweet"]], dtype=np.int32)

x_train.shape, x_valid.shape, x_test.shape

In [ ]:
#данные после подготовки к обучению
torch.from_numpy(x_train[0])

## 2. Dataset и Dataloader

In [ ]:
#Датасет
class DataWrapper(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data).long() #преобразуем в целочисленный тензор
        self.target = torch.from_numpy(target).long() #преобразуем в целочисленный тензор
        self.transform = transform

    def __getitem__(self, index):
        x = self.data[index]  #индексация данных
        y = self.target[index]  #индексация целевой переменной

        if self.transform:
            x = self.transform(x)

        return x, y

    def __len__(self):
        return len(self.data)

In [ ]:
#создание представителей классов Dataset и Dataloader
train_dataset = DataWrapper(x_train, df_train['label'].values)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

valid_dataset = DataWrapper(x_valid, df_valid['label'].values)
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=True)

In [ ]:
for x, l in train_loader:
    print(x.shape, x[0])
    print(l.shape, l[0])
    break

## 3. Построение и обучение модели LSTM

In [ ]:

#LSTM
class LSTMFixedLen(nn.Module):
    def __init__(self, vocab_size, embedding_dim=128, hidden_dim=128, use_last=True):
        super().__init__()
        self.use_last = use_last
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=2, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.embeddings(x)
        x = self.dropout(x)

        lstm_out, ht = self.lstm(x)
        # lstm_out: тензор с выходными фичами с последнего слоя для каждого t #(B, L, Hout)
        # h_t: тензор с последними скрытыми состояниями по слоям  #(num_layers, B, Hout)

        if self.use_last:
            last_tensor = lstm_out[:,-1,:]  #(B, Hout) тензор с последней строкой из каждого батча
        else:
            # use mean
            last_tensor = torch.mean(lstm_out[:,:], dim=1)  #(B, Hout) тензор со средним арифметическим столбцов по каждому батчу

        out = self.linear(last_tensor)

        return torch.sigmoid(out)

In [ ]:
#инициализация модели
model_lstm = LSTMFixedLen(max_words, 128, 128, use_last=False)
print(model_lstm)
print("Parameters:", sum([param.nelement() for param in model_lstm.parameters()]))

In [ ]:
#компиляция модели
optimizer = torch.optim.Adam(model_lstm.parameters(), lr=0.001)
criterion = nn.BCELoss()

In [ ]:
%%time

#обучение модели
model_lstm.train()

train_loss_history = []
valid_loss_history = []

train_acc_history_lstm = []
valid_acc_history_lstm = []
train_acc = torchmetrics.Accuracy(task = 'binary')
test_acc = torchmetrics.Accuracy(task = 'binary')

for epoch in range(epochs):
    model_lstm.train()
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()
        outputs = model_lstm(inputs)
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()

        # подсчет ошибки на обучении
        loss = loss.item()

        # подсчет метрики на обучении
        train_acc(outputs.squeeze(), labels)

    # выводим статистику о процессе обучения
    model_lstm.eval()

    print(f'Epoch [{epoch + 1}/{epochs}]. ' \
            f'Step [{i + 1}/{len(train_loader)}]. ' \
            f'Loss: {loss:.3f}. ' \
            f'Acc: {train_acc.compute().item():.3f}', end='. ')

    train_loss_history.append(loss)
    train_acc_history_lstm.append(train_acc.compute().item())
    train_acc.reset()


    # выводим статистику на тестовых данных
    for j, data in enumerate(valid_loader):
        test_labels = data[1]
        test_outputs = model_lstm(data[0])

        # подсчет ошибки на тесте
        test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))

        # подсчет метрики на тесте
        test_acc(test_outputs.squeeze(), test_labels)

    print(f'Test loss: {test_loss:.3f}. Test acc: {test_acc.compute().item():.3f}')
    valid_loss_history.append(test_loss.item())
    valid_acc_history_lstm.append(test_acc.compute().item())
    test_acc.reset()

print('Training is finished!')

In [ ]:
#визуализации функции потерь
plt.title('Loss history')
plt.grid(True)
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.plot(train_loss_history, label='train')
plt.plot(valid_loss_history, label='valid')
plt.legend()

In [ ]:
#визуализации метрики
plt.title('Accuracy history')
plt.grid(True)
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.plot(train_acc_history_lstm, label='train')
plt.plot(valid_acc_history_lstm, label='valid')
plt.legend()

## 4. Построение и обучение модели GRU

In [ ]:
#GRU
class GRUFixedLen(nn.Module):
    def __init__(self, vocab_size, embedding_dim=128, hidden_dim=128, use_last=True):
        super().__init__()
        self.use_last = use_last
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.gru = nn.GRU(embedding_dim, hidden_dim, num_layers=2, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.embeddings(x)
        x = self.dropout(x)

        gru_out, ht = self.gru(x)
        # gru_out: тензор с выходными фичами с последнего слоя для каждого t #(B, L, Hout)
        # h_t: тензор с последними скрытыми состояниями по слоям  #(num_layers, B, Hout)

        if self.use_last:
            last_tensor = gru_out[:,-1,:]  #(B, Hout) тензор с последней строкой из каждого батча
        else:
            # use mean
            last_tensor = torch.mean(gru_out[:,:], dim=1)  #(B, Hout) тензор со средним арифметическим столбцов по каждому батчу

        out = self.linear(last_tensor)

        return torch.sigmoid(out)

In [ ]:
#инициализация модели
model_gru = GRUFixedLen(max_words, 128, 128, use_last=False)
print(model_gru)
print("Parameters:", sum([param.nelement() for param in model_gru.parameters()]))

In [ ]:
#компиляция модели
optimizer = torch.optim.Adam(model_gru.parameters(), lr=0.001)
criterion = nn.BCELoss()

In [ ]:
%%time

#обучение модели
model_gru.train()

train_loss_history = []
valid_loss_history = []

train_acc_history_gru = []
valid_acc_history_gru = []

train_acc = torchmetrics.Accuracy('binary')
test_acc = torchmetrics.Accuracy('binary')

for epoch in range(epochs):
    model_gru.train()
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()
        outputs = model_gru(inputs)
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()

        # подсчет ошибки на обучении
        loss = loss.item()

        # подсчет метрики на обучении
        train_acc(outputs.squeeze(), labels)

    # выводим статистику о процессе обучения
    model_gru.eval()

    print(f'Epoch [{epoch + 1}/{epochs}]. ' \
            f'Step [{i + 1}/{len(train_loader)}]. ' \
            f'Loss: {loss:.3f}. ' \
            f'Acc: {train_acc.compute().item():.3f}', end='. ')

    train_loss_history.append(loss)
    train_acc_history_gru.append(train_acc.compute().item())
    train_acc.reset()


    # выводим статистику на тестовых данных

    for j, data in enumerate(valid_loader):
        test_labels = data[1]
        test_outputs = model_gru(data[0])

        # подсчет ошибки на тесте
        test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))

        # подсчет метрики на тесте
        test_acc(test_outputs.squeeze(), test_labels)

    print(f'Test loss: {test_loss:.3f}. Test acc: {test_acc.compute().item():.3f}')
    valid_loss_history.append(test_loss.item())
    valid_acc_history_gru.append(test_acc.compute().item())
    test_acc.reset()

print('Training is finished!')

In [ ]:
#визуализации функции потерь
plt.title('Loss history')
plt.grid(True)
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.plot(train_loss_history, label='train')
plt.plot(valid_loss_history, label='valid')
plt.legend()

In [ ]:
#визуализации метрики
plt.title('Accuracy history')
plt.grid(True)
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.plot(train_acc_history_gru, label='train')
plt.plot(valid_acc_history_gru, label='valid')
plt.legend()

## 5. Выводы

In [ ]:
# сводная таблица метрик моделей
metrics_result = pd.DataFrame([{'Acc_train': train_acc_history_lstm[-1],
                                'Acc_test': valid_acc_history_lstm[-1]},
                               {'Acc_train':  train_acc_history_gru[-1],
                                'Acc_test': valid_acc_history_gru[-1]}],
                              index =['LSTM', 'GRU'],
                              columns =['Acc_train', 'Acc_test'])

metrics_result.sort_values(by='Acc_test', ascending=False)

Выводы:

- Поскольку метрика на валидации растет немонотонно, то увеличение числа эпох не способствует росту метрик на валидации.

- Увеличение embedding_dim приводит к росту метрик на валидации.

- Увеличение hidden_dim способствует росту метрики на валидации.

- Увеличение max_words влечет увеличение метрики на валидации.

- При уменьшении max_len метрика на валидации растет.

- Пункты 2-4 увеличивают время обучения.

- При одних и тех же гиперпараметрах GRU обучается быстрее, чем LSTM. При этом метрики моделей примерно одинаковы.